<a href="https://colab.research.google.com/github/davidbau/rome/blob/gpt_stats/notebooks/statistics_of_GPT_states.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
cd /content && rm -rf /content/rome
git clone https://github.com/davidbau/rome -b gpt_stats rome > install.log 2>&1
pip install -r /content/rome/scripts/colab_reqs/rome.txt >> install.log 2>&1

In [ ]:
IS_COLAB = False
try:
    import google.colab, torch, sys
    if not torch.cuda.is_available():
        print("Change runtime type to include a GPU.")
    sys.path.append('/content/rome')
    IS_COLAB = True
except:
    pass

# Statistics of GPT states

Here we load GPT-2-XL and examine the statistics of its internal states.

We are particularly interested in: how close are the output values to Gaussian.

In [ ]:
%load_ext autoreload
%autoreload 2

We build upon the ROME codebase.

There is an `experiments.causal_trace` module that contains some useful utility functions for loading large transformer models in colab.

In [ ]:
import os, re, json, math
import torch, numpy
from collections import defaultdict
from experiments.causal_trace import ModelAndTokenizer, predict_token
from baukit import show, nethook

Now we load GPT-2-XL and a tokenizer, and show that it can complete a couple factual statements correctly.

In [ ]:
torch.set_grad_enabled(False)
mt = ModelAndTokenizer('gpt2-xl', low_cpu_mem_usage=IS_COLAB)


Now for some neutral text samples, we load some random texts from wikipedia.  Just a few (one batch for now).

In [ ]:
from datasets import load_dataset
from baukit import TokenizedDataset, length_collation, flatten_masked_batch

def get_ds(ds_name='wikipedia', tokenizer=None, batch_tokens=None):
    raw_ds = load_dataset(
        ds_name,
        dict(wikitext="wikitext-103-raw-v1", wikipedia="20200501.en")[ds_name],
    )
    maxlen = mt.model.config.n_positions
    if batch_tokens is not None and batch_tokens < maxlen:
        maxlen = batch_tokens
    return TokenizedDataset(raw_ds["train"], tokenizer, maxlen=maxlen)
ds = get_ds(tokenizer=mt.tokenizer, ds_name='wikipedia')

In [ ]:
mt.tokenizer.decode(ds[0]['input_ids'])

In [ ]:
if True:
    from baukit import show
    item = ds[0]
    cuda_batch = {k: v[None].cuda() for k, v in item.items()}
    logits = mt.model(**cuda_batch)['logits']
    probs = torch.nn.functional.softmax(logits, dim=2)
    input_ids = item['input_ids']
    preds = probs.argmax(dim=2)[0]
    baseline_probs = probs[0, torch.arange(len(input_ids) - 1), input_ids[1:]]
    show([[mt.tokenizer.decode(t), f'{p.item():.3g}', mt.tokenizer.decode(i), mt.tokenizer.decode(j)]
      for t, i, p, j in zip(input_ids[:-1], input_ids[1:], baseline_probs, preds)])

In [ ]:
# Now set up a second copy in which the layer 17 MLP layer is randomized
def make_perturbed_model():
    mt2 = ModelAndTokenizer('gpt2-xl', low_cpu_mem_usage=IS_COLAB)
    for layernum in range(10, 11):
        R = torch.randn(6400, 1600)
        RU, RS, RV = R.svd()
        orig_W = mt2.model.transformer.h[layernum].mlp.c_proj.weight.detach().cpu()
        U, S, V = orig_W.svd()
        randomized_W = U @ torch.diag(S) @ RV.t()
        mt2.model.transformer.h[layernum].mlp.c_proj.weight[...] = randomized_W
    return mt2

In [ ]:
mt2 = make_perturbed_model()

In [ ]:
noise_const = torch.from_numpy(numpy.random.RandomState(1).randn(1024 * 1024)).float().cuda()
PLAIN = show.style()
RED = show.style(background='red')
BLUE = show.style(background='skyblue')
HEADING = show.style(fontWeight='bold', borderBottom='1px solid black')

def filter_and_print(mt, input_ids, baseline_probs, preds, n_baseline_probs, n_preds, threshold=5.0, docid=None):
    gt = input_ids[1:].cuda()
    new_mistakes = (gt == preds[:-1]) & (gt != n_preds[:-1]) & (gt != 198) & (
        n_baseline_probs * threshold < baseline_probs)
    if not new_mistakes.sum():
        return
    if docid is not None:
        print(f'docid {docid}')
        
    show(
        [HEADING, ['tok', 'gtprob', 'corrupted', 'gt', 'gti', 'pred', 'corrupted']] +
         [[show.Tag('p'), mt.tokenizer.decode(t),
           f'{p.item():.3g}',
           (RED if np.item() * threshold < p.item() else PLAIN), f'{np.item():.3g}',
           (RED if new_mistakes[ind] else PLAIN), mt.tokenizer.decode(i),
           i.item(),
           mt.tokenizer.decode(j),
           mt.tokenizer.decode(nj)]
      for ind, (t, i, p, j, np, nj) in enumerate(
          zip(input_ids[:-1],
              input_ids[1:],
              baseline_probs,
              preds,
              n_baseline_probs,
              n_preds))
          if new_mistakes[ind]
         ]
    )
    
    buf = ['<pre>']
    for ind, t in enumerate(input_ids):
        if ind > 0 and new_mistakes[ind - 1]:
            buf.append('<b style="color:red">')
            buf.append(mt.tokenizer.decode(t))
            buf.append('</b>')
        else:
            buf.append(mt.tokenizer.decode(t))
    buf.append('</pre>')
    show(show.raw_html(''.join(buf)))


def run_perturbed(datum, threshold=5.0, docid=None):
    maxlen = mt.model.config.n_positions
    input_ids = datum['input_ids'][:maxlen]
    cuda_batch = {k: v[None][:maxlen].cuda() for k, v in datum.items()}
    logits = mt.model(**cuda_batch)['logits']
    probs = torch.nn.functional.softmax(logits, dim=2)
    preds = probs.argmax(dim=2)[0]
    baseline_probs = probs[0, torch.arange(len(input_ids) - 1), input_ids[1:]]
    n_logits = mt2.model(**cuda_batch)['logits']
    n_probs = torch.nn.functional.softmax(n_logits, dim=2)
    n_preds = n_probs.argmax(dim=2)[0]
    n_baseline_probs = n_probs[0, torch.arange(len(input_ids) - 1), input_ids[1:]]
    filter_and_print(mt, input_ids, baseline_probs, preds, n_baseline_probs, n_preds, threshold=threshold, docid=docid)
    
def old_filter_and_print(mt, input_ids, baseline_probs, preds, n_baseline_probs, n_preds, threshold=5.0, docid=None):
    gt = input_ids[1:].cuda()
    new_mistakes = (gt == preds[:-1]) & (gt != n_preds[:-1])
    #if new_mistakes.sum() and (baseline_probs / n_baseline_probs).max() > threshold:
    if new_mistakes.sum() and (baseline_probs[new_mistakes] / n_baseline_probs[new_mistakes]).max() > threshold:
        last_index = ((baseline_probs / n_baseline_probs) > threshold).nonzero()[-1, 0] + 5
        show.bare(show.style(display='flex'), [
            [HEADING, ['tok', 'gtprob', 'corrupted', 'gt', 'pred', 'corrupted']] +
             [[show.Tag('p'), mt.tokenizer.decode(t),
               f'{p.item():.3g}',
               (RED if np.item() * threshold < p.item() else PLAIN), f'{np.item():.3g}',
               (RED if new_mistakes[ind] else PLAIN), mt.tokenizer.decode(i),
               mt.tokenizer.decode(j),
               mt.tokenizer.decode(nj)]
          for ind, (t, i, p, j, np, nj) in enumerate(
              zip(input_ids[:-1],
                  input_ids[1:last_index],
                  baseline_probs,
                  preds,
                  n_baseline_probs,
                  n_preds))]
        ])
#run_perturbed(ds[7], threshold=2.0, docid=7)

In [ ]:
for doc in range(0, 10000):
    run_perturbed(ds[doc], threshold=5.0, docid=doc)